<font size="5">**Data Science Section 002 Group 7 Project Report**

<font size="3">**Research question:** How do the variables age, resting blood pressure, serum cholesterol, and maximum heart rate achieved affect the accuracy of a knn-classification model predicting the presence or absence of heart disease in individuals?

Predictive data analysis plays an important role in the field of medical diagnostics. As cardiovascular disease continues to be a leading cause of global mortality, the development of efficient predictive models becomes crucial for detection and intervention. This analysis focuses on predicting the presence or absence of heart disease using key parameters from the Heart Disease dataset (Cleveland database) available on the UCI Machine Learning Repository. The original database contains 76 attributes, but a version containing a subset of only 14 of those attributes will be used for this analysis. Out of those 14 variables, only 5 are chosen for observation in our project: age (age), resting blood pressure (trestbps), serum cholesterol (chol), maximum heart rate achieved (thalach), and our predicted variable, the diagnosis of heart disease (num, renamed "diagnosis").

To further understand our data, some preliminary exploratory data analysis is conducted below. To gain access to useful functions for reading and analyzing our data, the following libraries must be installed:

In [2]:
library(tidyverse)
library(tidymodels)

Warning message:
“package ‘ggplot2’ was built under R version 4.3.2”
── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.3     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.0
✔ ggplot2   3.5.0     ✔ tibble    3.2.1
✔ lubridate 1.9.2     ✔ tidyr     1.3.0
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
── Attaching packages ────────────────────────────────────── tidymodels 1.1.1 ──

✔ broom        1.0.5     ✔ rsample      1.2.0
✔ dials        1.2.0     ✔ tune         1.1.2
✔ infer        1.0.5     ✔ workflows    1.1.3
✔ modeldata    1.2.0     ✔ workflowsets 1.0.1
✔ parsnip      1.1.1     ✔ yardstick    1.2.0
✔ recipes      1.0.8     

── Conflicts ───────────────────────────────────────── tidymodels_co

We can now read our dataset into R and assign it to an object called cleveland_data. We are also going to limit the amount of rows shown when previewing the dataset:

In [3]:
#Demostration that the dataset can be read from the web into R:
options(repr.matrix.max.rows = 10)

url <- "https://raw.githubusercontent.com/Nitro-87/Group-2-Project-Proposal/main/processed.cleveland.data"
cleveland_data <- read_delim(url, delim= "," , col_names=FALSE)

cleveland_data

Rows: 303 Columns: 14
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr  (2): X12, X13
dbl (12): X1, X2, X3, X4, X5, X6, X7, X8, X9, X10, X11, X14

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,X11,X12,X13,X14
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<dbl>
63,1,1,145,233,1,2,150,0,2.3,3,0.0,6.0,0
67,1,4,160,286,0,2,108,1,1.5,2,3.0,3.0,2
67,1,4,120,229,0,2,129,1,2.6,2,2.0,7.0,1
37,1,3,130,250,0,0,187,0,3.5,3,0.0,3.0,0
41,0,2,130,204,0,2,172,0,1.4,1,0.0,3.0,0
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
45,1,1,110,264,0,0,132,0,1.2,2,0.0,7.0,1
68,1,4,144,193,1,0,141,0,3.4,2,2.0,7.0,2
57,1,4,130,131,0,0,115,1,1.2,2,1.0,7.0,3


Since our data contains no column names, we rename each column with the appropriate variable name below. We also replace the numerical values under our categorical variables with what they are supposed to represent, to better understand what each variable reveals to us.

In [4]:
#Renaming columns:

cleveland_data <- rename(cleveland_data,
       age = X1,
       sex = X2,
       chest_pain_type = X3,
       trestbps = X4,
       chol = X5,
       fbs = X6,    
       restecg = X7,
       thalach = X8,
       exang = X9,
       oldpeak = X10,
       slope = X11,
       ca = X12,
       thal = X13,
       diagnosis = X14)

#skipping rows with missing values
cleveland_data<- cleveland_data |> drop_na()

glimpse(cleveland_data)

Rows: 303
Columns: 14
$ age             <dbl> 63, 67, 67, 37, 41, 56, 62, 57, 63, 53, 57, 56, 56, 44…
$ sex             <dbl> 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, …
$ chest_pain_type <dbl> 1, 4, 4, 3, 2, 2, 4, 4, 4, 4, 4, 2, 3, 2, 3, 3, 2, 4, …
$ trestbps        <dbl> 145, 160, 120, 130, 130, 120, 140, 120, 130, 140, 140,…
$ chol            <dbl> 233, 286, 229, 250, 204, 236, 268, 354, 254, 203, 192,…
$ fbs             <dbl> 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, …
$ restecg         <dbl> 2, 2, 2, 0, 2, 0, 2, 0, 2, 2, 0, 2, 2, 0, 0, 0, 0, 0, …
$ thalach         <dbl> 150, 108, 129, 187, 172, 178, 160, 163, 147, 155, 148,…
$ exang           <dbl> 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, …
$ oldpeak         <dbl> 2.3, 1.5, 2.6, 3.5, 1.4, 0.8, 3.6, 0.6, 1.4, 3.1, 0.4,…
$ slope           <dbl> 3, 2, 2, 3, 1, 1, 3, 1, 2, 3, 2, 2, 2, 1, 1, 1, 3, 1, …
$ ca              <chr> "0.0", "3.0", "2.0", "0.0", "0.0", "0.0", "2.0", "0.0"…
$ thal            

In [ ]:
#renaming categorical variables:

#renaming sex values
cleveland_data <- cleveland_data |>
mutate(sex = as_factor(sex))|>
mutate(sex = fct_recode(sex, "female" = "0", 
                            "male" = "1"))
#renaming chest pain values
cleveland_data <- cleveland_data |>
mutate(chest_pain_type = as_factor(chest_pain_type))|>
mutate(chest_pain_type = fct_recode(chest_pain_type, 
                            "typical angina" = "1", 
                            "atypical angina" = "2", 
                            "non-anginal_pain" = "3", 
                            "asymptomatic" = "4"))
#renaming fbs values
cleveland_data <- cleveland_data |>
mutate(fbs = as_factor(fbs))|>
mutate(fbs = fct_recode(fbs, "false" = "0", 
                            "true" = "1"))

#renaming exang values
cleveland_data <- cleveland_data |>
mutate(exang = as_factor(exang))|>
mutate(exang = fct_recode(exang, "no" = "0", 
                            "yes" = "1"))
#renaming slope values
cleveland_data <- cleveland_data |>
mutate(slope = as_factor(slope))|>
mutate(slope = fct_recode(thal,"upsloping" = "1", 
                            "flat" = "2", 
                            "downsloping" = "3"))
#renaming thal values
cleveland_data <- cleveland_data |>
mutate(thal = as_factor(thal))|>
mutate(thal = fct_recode(thal, "reversible defect" = "7.0", 
                            "fixed defect" = "6.0", 
                            "normal" = "3.0"))
#renaming diagnosis values
cleveland_data <- cleveland_data |>
mutate(diagnosis = as_factor(diagnosis))|>
mutate(diagnosis = fct_recode(diagnosis, 
                            "absent" = "0",
                              "present" = "1", 
                            "present" = "2", 
                            "present" = "3", 
                            "present" = "4"))

glimpse(cleveland_data)

We then want to filter our dataset to only include the diagnosis and variables that we want to observe which are age, chol, tresbps and thalach.

In [ ]:
cleveland_data_select <- select(cleveland_data, age, trestbps, chol, thalach, diagnosis)
cleveland_data_select

Now that our data is tidy and filtered, we can split it into a training set to construct our model, and a testing set to test our model's effectiveness.

In [ ]:
#splitting data into training and testing sets

cleveland_data_split <- initial_split(cleveland_data_select, prop = 0.75, strata = diagnosis)
training_set <- training(cleveland_data_split)
testing_set <- testing(cleveland_data_split)

training_set
testing_set

Below is a table conducting exploratory analysis on our training dataset. This table tells us the number and percentage of observations in each class group (whether there is a presence/absence of heart disease).

In [ ]:
num_obs <- nrow(training_set)
training_set |>
  group_by(diagnosis) |>
  summarize(
    count = n(),
    percentage = n() / num_obs * 100)

We can also use the training dataset to create a set of histograms to show us the distributions and values for each variable.

In [ ]:
options(repr.plot.width = 9, repr.plot.height = 8)
age_hist <- ggplot(training_set, aes(x = age)) +
                geom_histogram(bins = 40) + 
                labs(x = "Age", y = "Count", title = "Distribution of Age of Patients")

thalach_hist <- ggplot(training_set, aes(x = thalach)) +
                geom_histogram(bins = 40) + 
                labs(x = "Maximum Heart Rate Achieved", y = "Count", title = "Distribution of Maximum Heart Rate Achieved of Patients")

chol_hist <- ggplot(training_set, aes(x = chol)) +
                geom_histogram(bins = 40) + 
                labs(x = "Serum Cholesterol (mg/dl)", y = "Count", title = "Distribution of Serum Cholesterol of Patients")

trestbps_hist <- ggplot(training_set, aes(x = trestbps)) +
                geom_histogram(bins = 40) + 
                labs(x = "Resting Blood Pressure", y = "Count", title = "Distribution of Resting Blood Pressure of Patients")

gridExtra::grid.arrange(age_hist, trestbps_hist, chol_hist, thalach_hist, ncol = 2)

We have chosen to use the K-nearest neighbors classification algorithm to continue our analysis, as it allows us to scale and compare our different variables in an easy and efficient way. The goal is to create and test a model using the training and testing data, respectively. We will be using this model to analyze the data in relation to the selected variables, and aim to effectively classify our testing data.

By the end of this analysis, we expect to gain insight into the complex relationships between our chosen variables and the presence of heart disease in patients. Such findings would contribute to medical research regarding cardiovascular disease by allowing healthcare systems to identify and focus on individuals at higher risk of heart diseases, enabling earlier intervention, and potentially reducing healthcare costs associated with advanced stages of heart disease. The analysis also prompts various questions for future analyses, such as exploring the interaction between lifestyle factors (smoking habits, exercise, and diet) and our chosen variables of cholesterol, age, heart rate and blood pressure in influencing the risk of heart disease.

The goal of our analysis is to see which of the above variables in combination are most affective at predicting the presence of heart disease in patients. To do so, we will utilize cross validation for each variable, to build a KNN model for each variable, so that we may compare their effectiveness on a subset of our training data. Afterwards we will use the indicators we deemed effective for our KNN model on our testing data, and compare this to the efficacy of a KNN model using all of the four variables. In our analysis we will prioritize recall and accuracy, because of the possibly life affecting nature of heart disease which may cause heart attack or stroke. $\color{red}{\text{citation needed}}$

To achieve this, we will need to split our training data again, so that we can use a portion of it to test our classifiers on each variable.

In [ ]:
training_split <- initial_split(training_set,prop = 0.75, strata = diagnosis)
training_subset <- training(training_split)
testing_subset <- testing(training_split)

Since we are doing the same analysis on each variable, we will do it side by side in the same code cells, as the code will be almost identical. We will test up to a relatively high $k$ value, since class imbalance won't be an issue.

In [ ]:
#This model will be used for all classifications
knn_tune <- nearest_neighbor(weight_func = "rectangular", neighbors = tune()) |> set_engine("kknn") |> set_mode("classification")

age_recipe <- recipe(diagnosis ~ age, data=training_subset) |> step_scale(all_predictors()) |> step_center(all_predictors())
heart_rate_recipe <-  recipe(diagnosis ~ thalach, data=training_subset) |> step_scale(all_predictors()) |> step_center(all_predictors())
cholesterol_recipe <- recipe(diagnosis ~ chol, data=training_subset) |> step_scale(all_predictors()) |> step_center(all_predictors())
blood_pressure_recipe <- recipe(diagnosis ~ trestbps, data=training_subset) |> step_scale(all_predictors()) |> step_center(all_predictors())

training_vfold = vfold_cv(training_subset,v = 5, strata = diagnosis)
kvals <- tibble(neighbors = seq(from = 2, to = 16, by = 2))

knn_age <- workflow() |> add_recipe(age_recipe) |> add_model(knn_tune) |> tune_grid(resamples = training_vfold, grid = kvals) |> collect_metrics()
knn_hr <- workflow() |> add_recipe(heart_rate_recipe) |> add_model(knn_tune) |> tune_grid(resamples = training_vfold, grid = kvals) |> collect_metrics()
knn_chol <- workflow() |> add_recipe(cholesterol_recipe) |> add_model(knn_tune) |> tune_grid(resamples = training_vfold, grid = kvals) |> collect_metrics()
knn_bp <- workflow() |> add_recipe(blood_pressure_recipe) |> add_model(knn_tune) |> tune_grid(resamples = training_vfold, grid = kvals) |> collect_metrics()
age_acc <- knn_age |> filter(.metric == "accuracy") |> select(neighbors, mean) |> mutate(variable = "age")
hr_acc <- knn_hr |> filter(.metric == "accuracy") |> select(neighbors, mean) |> mutate(variable = "hr")
chol_acc <- knn_chol |> filter(.metric == "accuracy") |> select(neighbors, mean) |> mutate(variable = "chol")
bp_acc <- knn_bp |> filter(.metric == "accuracy") |> select(neighbors, mean) |> mutate(variable = "bp")
#Now we will combine the accuracies into a single data set for plotting
acc <- bind_rows(age_acc,hr_acc, chol_acc, bp_acc)
best_k_age <- age_acc |> arrange(mean) |> slice(8) |> select(neighbors) |> pull()
best_k_hr <- hr_acc |> arrange(mean) |> slice(8) |> select(neighbors) |> pull()
best_k_chol <- chol_acc |> arrange(mean) |> slice(8) |> select(neighbors) |> pull()
best_k_bp <- bp_acc |> arrange(mean) |> slice(8) |> select(neighbors) |> pull()

Now we can compare the effectiveness of each $k$ value on predictions using each individual variable, so that we can compare the efficacies using th best $k$-values for each variable.

In [ ]:
options(repr.plot.width = 6, repr.plot.height = 3)
#I will include a plot here color coding each variable, and plotting all variables accuracy vs k
cross_val_plot <- acc |> ggplot(aes(x=neighbors,y=mean)) + geom_line(aes(colour = variable)) + 
labs(x="neighbors", y="accuracy") + ggtitle("How number of neighbors affects accuracy")
cross_val_plot

Now we can compare how effective our predictors are on the testing subset of our training data, in order to choose which predictors we would like to use for our final model.

In [ ]:
knn_spec_age <- nearest_neighbor(weight_func = "rectangular", neighbors = best_k_age) |> set_engine("kknn") |> set_mode("classification")
knn_spec_hr <- nearest_neighbor(weight_func = "rectangular", neighbors = best_k_hr) |> set_engine("kknn") |> set_mode("classification")
knn_spec_bp <- nearest_neighbor(weight_func = "rectangular", neighbors = best_k_bp) |> set_engine("kknn") |> set_mode("classification")
knn_spec_chol <- nearest_neighbor(weight_func = "rectangular", neighbors = best_k_chol) |> set_engine("kknn") |> set_mode("classification")

age_fit <- workflow() |> add_recipe(age_recipe) |> add_model(knn_spec_age) |> fit(training_subset)
hr_fit <- workflow() |> add_recipe(heart_rate_recipe) |> add_model(knn_spec_hr) |> fit(training_subset)
bp_fit <- workflow() |> add_recipe(blood_pressure_recipe) |> add_model(knn_spec_bp) |> fit(training_subset)
chol_fit <- workflow() |> add_recipe(cholesterol_recipe) |> add_model(knn_spec_chol) |> fit(training_subset)

age_pred <- predict(age_fit,testing_subset) |> bind_cols(testing_subset)
hr_pred <- predict(hr_fit,testing_subset) |> bind_cols(testing_subset)
bp_pred <- predict(bp_fit,testing_subset) |> bind_cols(testing_subset)
chol_pred <- predict(chol_fit,testing_subset) |> bind_cols(testing_subset)

#here we use the formulas to compute recall/accuracy for each predictor
age_acc <- nrow(filter(age_pred, .pred_class == diagnosis))/nrow(age_pred)
age_recall <- nrow(filter(age_pred, .pred_class == "present" & diagnosis == "present"))/nrow(filter(age_pred,diagnosis == "present"))
hr_acc <- nrow(filter(hr_pred, .pred_class == diagnosis))/nrow(hr_pred)
hr_recall <- nrow(filter(hr_pred, .pred_class == "present" & diagnosis == "present"))/nrow(filter(hr_pred,diagnosis == "present"))
bp_acc <- nrow(filter(bp_pred, .pred_class == diagnosis))/nrow(bp_pred)
bp_recall <- nrow(filter(bp_pred, .pred_class == "present" & diagnosis == "present"))/nrow(filter(bp_pred,diagnosis == "present"))
chol_acc <- nrow(filter(chol_pred, .pred_class == diagnosis))/nrow(chol_pred)
chol_recall <- nrow(filter(chol_pred, .pred_class == "present" & diagnosis == "present"))/nrow(filter(chol_pred,diagnosis == "present"))

#Now as a tibble
var_names <- as_tibble(c("age", "age", "max heart rate", "max heart rate", "resting blood pressure","resting blood pressure", "serum cholesterol", "serum cholesterol")) |> rename(variable = value)
metric_type <- as_tibble(c("accuracy", "recall","accuracy", "recall","accuracy", "recall","accuracy", "recall")) |> rename(metric = value)
value <- as_tibble(c(age_acc,age_recall,hr_acc,hr_recall,bp_acc,bp_recall,chol_acc,chol_recall))     
recall_acc_preds <- bind_cols(var_names,metric_type,value)
recall_preds <- recall_acc_preds |> filter(metric == "recall")
acc_preds <- recall_acc_preds |> filter(metric == "accuracy")

Here are plots comparing the accuracy and recall of a model when using each predictor respectively

In [1]:
accuracy_plot <- ggplot(acc_preds, aes(x = variable, y = value)) + geom_bar(stat = "identity", aes(fill = variable)) + labs(x = "predictor", y = "accuracy")

accuracy_plot

ERROR: Error in ggplot(acc_preds, aes(x = variable, y = value)): could not find function "ggplot"
